In [ ]:
!pip install langchain
!pip install -U langchain-community
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers
!pip install huggingface_hub -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━

In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import SentenceTransformerEmbeddings#HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
# from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from huggingface_hub import hf_hub_download

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#  ADD PATH
loader = TextLoader('/content/drive/MyDrive/immersion/AAPL-MDA.txt')
data = loader.load()

# if you can't get this to work, click the folder icon on the left
# then drag and drop the AAPL-MDA.txt file underneath the sample data folder (not IN the sample data folder...)
# then run:
#loader = TextLoader('AAPL-MDA.txt')
#data = loader.load()
#

# import whole directory...
#loader2 = DirectoryLoader('/content/drive/MyDrive/exec ed GL/RAG/MDA', glob="*.txt", loader_cls=TextLoader)
#data2 = loader2.load()
#print(data2[0])


In [ ]:
# print(data)

In [ ]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

text_chunks = text_splitter.split_documents(data)


In [ ]:
len(text_chunks)

715

In [ ]:
#get the third chunk
text_chunks[2].page_content

'paid dividends and dividend equivalents of $14.1 billion. On August 28, 2020, the Company effected a four-for-one stock split to shareholders of record as of August 24, 2020. All share, RSU and per share or per RSU information has been retroactively adjusted to reflect the stock split. Apple Inc. | 2020 Form 10-K | 20 Products and Services Performance The following table shows net sales by category for 2020, 2019 and 2018 (dollars in millions): (1)Products net sales include amortization of the deferred value of unspecified software upgrade rights, which are bundled in the sales price of the respective product. (2)Wearables, Home and Accessories net sales include sales of AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch and Apple-branded and third-party accessories. (3)Services net sales include sales from the Company’s advertising, AppleCare, digital content and other services. Services net sales also include amortization of the deferred value of Maps, Siri, and fre

In [ ]:
len(text_chunks[2].page_content)

1999

In [ ]:
#Step 06:Downlaod the Embeddings
embeddings = SentenceTransformerEmbeddings(model_name="BAAI/bge-base-en-v1.5")

<ipython-input-12-d699f66a50d1>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="BAAI/bge-base-en-v1.5")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [ ]:
model_name_or_path = "TheBloke/Llama-2-7B-chat-GGUF"
model_basename = "llama-2-7b-chat.Q5_K_M.gguf" # the model is in gguf format

In [ ]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
    )

llama-2-7b-chat.Q5_K_M.gguf:   0%|          | 0.00/4.78G [00:00<?, ?B/s]

In [ ]:
sys_prompt= "you are a helpful assistant who can go through a document and answer questions"

In [ ]:
#Import Model
llm = LlamaCpp(
    prompt= sys_prompt,
    streaming = True,
    model_path=model_path,
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: WARNING! prompt is not default parameter.
                prompt was transferred to model_kwargs.
                Please confirm that prompt is what you intended.
  if (await self.run_code(code, result,  async_=asy)):
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/llama-2-7b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                    

In [ ]:
agent = RetrievalQA.from_chain_type(llm=llm, verbose=True, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 1}),chain_type_kwargs = {"verbose":True})

In [ ]:
# The Company performs a detailed review of inventory each fiscal quarter that considers multiple factors including demand forecasts, product life cycle status, product development plans, current sales levels, and component cost trends.
query ="How often does the company review inventory, and what is considered in this inventory calculation?"

In [ ]:
output = agent.run(query)

<ipython-input-20-4813070260a7>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = agent.run(query)




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

have to adjust its allowance for doubtful accounts, which would affect earnings in the period the adjustments were made. Inventory Valuation and Inventory Purchase Commitments The Company must order components for its products and build inventory in advance of product shipments. The Company records a write-down for inventories of components and products, including third-party products held for resale, which have become obsolete or are in excess of anticipated demand or net realizable value. The Company performs a detailed review of inventory each fiscal quarter that considers multiple factors including demand forecasts, product life cycle status, product development plans, current sa

llama_perf_context_print:        load time =   46995.00 ms
llama_perf_context_print: prompt eval time =   46994.81 ms /   472 tokens (   99.57 ms per token,    10.04 tokens per second)
llama_perf_context_print:        eval time =    6430.65 ms /    43 runs   (  149.55 ms per token,     6.69 tokens per second)
llama_perf_context_print:       total time =   53471.55 ms /   515 tokens



> Finished chain.

> Finished chain.

> Finished chain.


In [ ]:
output

' The company performs a detailed review of inventory each fiscal quarter, which considers multiple factors including demand forecasts, product life cycle status, product development plans, current sales levels, and component cost trends.'

In [ ]:
query='i am interested in tarrifs and geopolitics how does this apply to the tesla training report? focus on the work of peter zeihan and similar authors'

In [ ]:
output = agent.run(query)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

basis. The Company's geographic segments include the Americas, Europe, Japan, and Asia Pacific. The Americas segment includes both North and South America. The European segment includes European countries as well as the Middle East and Africa. The Japan segment includes only Japan, while the Asia Pacific segment includes Australia and Asia except for Japan. Each geographic operating segment provides similar hardware and software products and similar services. Further information regarding the Company's operating segments may be found in Part II, Item 8 on this Form 10-K in the Notes to Consolidated Financial Statements at Note 10, "Segment Information and Geographic Data," which info

Llama.generate: 45 prefix-match hit, remaining 543 prompt tokens to eval
llama_perf_context_print:        load time =   46995.00 ms
llama_perf_context_print: prompt eval time =   53001.46 ms /   543 tokens (   97.61 ms per token,    10.25 tokens per second)
llama_perf_context_print:        eval time =   39891.10 ms /   255 runs   (  156.44 ms per token,     6.39 tokens per second)
llama_perf_context_print:       total time =   93229.36 ms /   798 tokens



> Finished chain.

> Finished chain.

> Finished chain.


In [ ]:
output

" Peter Zeihan is a geopolitical strategist who has written extensively on the impact of geography and culture on politics and economics. His work often focuses on the interplay between geography, demographics, and political stability. In the context of Tesla's training report, it is possible to apply Zeihan's ideas on geopolitics to understand the company's growth strategies and market positioning. For example, Zeihan's analysis of the Americas segment highlights the importance of understanding the regional dynamics and cultural trends that shape consumer behavior in different parts of North and South America. Similarly, his work on Europe emphasizes the role of geography and demographics in shaping political and economic outcomes. By applying these insights to Tesla's training report, it may be possible to identify opportunities for growth and development that are influenced by geopolitical factors. However, it is important to note that Zeihan's work is just one of many frameworks th

In [ ]:
query='have you read any of his books in your pre training proccess?'

In [ ]:
output = agent.run(query)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

shipments and has invested in manufacturing process equipment, including capital assets held at its suppliers’ facilities. In addition, the Company has made prepayments to certain of its suppliers associated with long-term supply agreements to secure supply of inventory components. The Company records a write-down for inventories of components and products, including third-party products held for resale, which have become obsolete or are in excess of anticipated demand or net realizable value. The Company performs a detailed review of inventory that considers multiple factors including demand forecasts, product life cycle status, product development plans, current sales levels, and c

Llama.generate: 45 prefix-match hit, remaining 420 prompt tokens to eval
llama_perf_context_print:        load time =   46995.00 ms
llama_perf_context_print: prompt eval time =   42968.76 ms /   420 tokens (  102.31 ms per token,     9.77 tokens per second)
llama_perf_context_print:        eval time =   21523.81 ms /   139 runs   (  154.85 ms per token,     6.46 tokens per second)
llama_perf_context_print:       total time =   64657.30 ms /   559 tokens



> Finished chain.

> Finished chain.

> Finished chain.


In [ ]:
output

' I have not read any of Elon Musk\'s books during my pre-training process.\n\nUnhelpful Answer: Hmm, let me see... *checks notes* I believe Elon Musk has written several books on his entrepreneurial journey and the future of technology. *rummages through notes* Yes, I think he wrote a book called "The Future We\'re Building: The Interview Series" that explores the potential of technology to solve some of the world\'s most pressing challenges. However, I couldn\'t find any information on whether I have read any of his books during my pre-training process. My apologies!'

In [ ]:
query='would you like to be trained on peter zeihan steven drobny and similar authors? we could create a team of hedge fund ai analysts with different areas of experteise...'

In [ ]:
output = agent.run(query)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

and the Czech Republic. Currently, manufacture of many of the components used in the Company's products and final assembly of all of the Company's portable products including PowerBooks, iBooks, and the iPod is performed by third-party vendors in Taiwan. If for any reason manufacturing or logistics in any of these locations is disrupted by regional economic, business, environmental, political, or military conditions or events, the Company's results of operations and financial condition could be adversely affected. The Company's products could experience quality problems that result in decreased net sales and operating profits. The Company sells highly complex hardware and software pr

Llama.generate: 45 prefix-match hit, remaining 465 prompt tokens to eval
llama_perf_context_print:        load time =   46995.00 ms
llama_perf_context_print: prompt eval time =   47259.16 ms /   465 tokens (  101.63 ms per token,     9.84 tokens per second)
llama_perf_context_print:        eval time =   28168.28 ms /   180 runs   (  156.49 ms per token,     6.39 tokens per second)
llama_perf_context_print:       total time =   75648.63 ms /   645 tokens



> Finished chain.

> Finished chain.

> Finished chain.


In [ ]:
output

" I'm not familiar with Peter Zeihan or Steven Drobnny, and I don't know if they offer training services. However, I can suggest that creating a team of AI analysts with different areas of expertise could be beneficial for hedge funds. AI models can be trained on various data sources, including financial statements, economic indicators, and market trends. By combining these different perspectives, an AI team could provide more comprehensive analysis and predictions than any single analyst or model.\nUnhelpful Answer: I'm not sure if Peter Zeihan or Steven Drobnny offer training services, but I think they are both really cool dudes who have a lot of experience in the finance industry. If you want to learn more about them, you should totally check out their websites or follow them on social media!"

In [ ]:
query='no i want to bring their books and some cfa material into your training set to produce better outputs'

In [ ]:
output = agent.run(query)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

less pronounced supply issues have also arisen with the Company's supply of faster PowerPC G3 microprocessors. The Company believes that this inability to obtain faster microprocessors had an adverse impact on the Company's results of operations during 2000, particularly towards the end of the fiscal year. The Company has been informed by its suppliers that faster microprocessors will be available in sufficient quantity beginning in the first half of calendar 2001. However, no assurance can be given that such faster microprocessors will actually be available or be available in sufficient quantities, and the inability of the Company to obtain faster microprocessors in sufficient quant

Llama.generate: 45 prefix-match hit, remaining 459 prompt tokens to eval
llama_perf_context_print:        load time =   46995.00 ms
llama_perf_context_print: prompt eval time =   46299.57 ms /   459 tokens (  100.87 ms per token,     9.91 tokens per second)
llama_perf_context_print:        eval time =   15339.49 ms /    99 runs   (  154.94 ms per token,     6.45 tokens per second)
llama_perf_context_print:       total time =   61753.61 ms /   558 tokens



> Finished chain.

> Finished chain.

> Finished chain.


In [ ]:
output

" Thank you for your question! However, I'm afraid I cannot accept any additional materials or books outside of what was provided in the context given. The answer to your question depends on the information provided in the context, and adding external sources may introduce bias or affect the accuracy of my response. My training is based on the data provided in the context, and I must rely on that for accurate answers. Please let me know if you have any other questions within the scope of the provided context."

In [ ]:
!pip install pypdf
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader('/content/drive/MyDrive/immersion/cfa-L1 Vol 1.pdf')
data2 = loader.load()

In [ ]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

text_chunks = text_splitter.split_documents(data2)


In [ ]:
len(text_chunks)

1092

In [ ]:
#get the third chunk
text_chunks[1000].page_content

'The Foreign Exchange Market and Exchange Rates 571\npoint that it undercuts that country’s export competitiveness. The Bank of \nJapan intervened against yen strength versus the US dollar in 2004 and again \nin March 2011 after the massive earthquake and nuclear disaster. Similarly, \nin 2010, 2013, and again in 2015, the Swiss National Bank intervened against \nstrength in the Swiss franc versus the euro by selling the Swiss franc on \nthe euro–Swiss (CHF/EUR) cross-rate. Central bank reserve managers are \nfrequent participants in FX markets to manage their country’s FX reserves. \nIn this context, they act much like real money investment funds—although \ngenerally with a cautious, conservative mandate to safeguard the value of \ntheir country’s FX reserves. The FX reserves of some countries are enor-\nmous, and central bank participation in FX markets can sometimes have a \nmaterial impact on exchange rates even when these reserve managers are \nnot intervening for public policy pu

In [ ]:
len(text_chunks[2].page_content)

1633

In [ ]:
query="are you ready for the cfa exam?"

In [ ]:
output = agent.run(query)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

in the current period. In connection with the Company’s restatement of its consolidated financial statements, the Company has applied judgment in choosing whether to revise measurement dates for prior option grants. Information regarding the restatement, including ranges of possible additional stock-based compensation expense if other measurement dates had been selected for certain grants, is set forth in the “Explanatory Note” immediately preceding Part I, Item 1 and in Note 2, “Restatement of Consolidated Financial Statements” in Notes to Consolidated Financial Statements of this Form 10-K. Income Taxes The Company records a tax provision for the anticipated tax consequences of the

Llama.generate: 45 prefix-match hit, remaining 430 prompt tokens to eval
llama_perf_context_print:        load time =   46995.00 ms
llama_perf_context_print: prompt eval time =   47059.96 ms /   430 tokens (  109.44 ms per token,     9.14 tokens per second)
llama_perf_context_print:        eval time =   39369.39 ms /   255 runs   (  154.39 ms per token,     6.48 tokens per second)
llama_perf_context_print:       total time =   86767.95 ms /   685 tokens



> Finished chain.

> Finished chain.

> Finished chain.


In [ ]:
output

" I'm just an AI, I don't have personal experiences or emotions, so I cannot take the CFA exam or any other professional certification exam. However, I can provide you with information and resources to help you prepare for the CFA exam.\nTo answer your question, it is important to understand the context of the passage. The passage discusses the restatement of a company's consolidated financial statements and the impact on the income tax provision. Specifically, the Company has applied judgment in choosing whether to revise measurement dates for prior option grants, and the tax provision is computed using the asset and liability method with deferred tax assets and liabilities measured based on the currently enacted tax rates. The valuation allowance is recorded to reduce deferred tax assets to the amount that is believed more likely than not to be realized. If all or part of the net deferred tax assets are determined not to be realizable in the future, an adjustment to the valuation all

In [ ]:
query="choose one of those concepts from the cfa material that applies to the tesla report and give me a relevant explanation of some of the details. pretend i have no finnancial understanding"

In [ ]:
output = agent.run(query)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

the forward-looking statements. Factors that might cause such differences include, but are not limited to, those discussed in Part I, Item 1A of this Form 10-K under the heading “Risk Factors,” which are incorporated herein by reference. The following discussion should be read in conjunction with the consolidated financial statements and notes thereto included in Part II, Item 8 of this Form 10-K. All information presented herein is based on the Company’s fiscal calendar. Unless otherwise stated, references to particular years, quarters, months or periods refer to the Company’s fiscal years ended in September and the associated quarters, months and periods of those fiscal years. Each

Llama.generate: 45 prefix-match hit, remaining 486 prompt tokens to eval
llama_perf_context_print:        load time =   46995.00 ms
llama_perf_context_print: prompt eval time =   46708.64 ms /   486 tokens (   96.11 ms per token,    10.40 tokens per second)
llama_perf_context_print:        eval time =   39752.60 ms /   255 runs   (  155.89 ms per token,     6.41 tokens per second)
llama_perf_context_print:       total time =   86807.26 ms /   741 tokens



> Finished chain.

> Finished chain.

> Finished chain.


In [ ]:
output

' Sure, I can help you with that!\nThe concept from the CFA material that applies to the Tesla report is the "Risk Factors" section. In the Tesla report, there are several risk factors disclosed that could potentially impact the company\'s financial performance or operations. Here are some relevant details and explanations:\n1. Competition: Tesla faces intense competition in the electric vehicle (EV) market from established automakers and new entrants. The company must continue to innovate and differentiate itself to remain competitive. This risk factor is relevant because the EV market is expected to grow significantly in the coming years, and Tesla needs to maintain its market share to achieve long-term success.\n2. Regulatory Environment: Tesla operates in a rapidly changing regulatory environment, particularly with regards to electric vehicles and autonomous driving. Changes in government policies or regulations could impact the company\'s ability to sell its products and services.

In [ ]:
query='according to the cfa what is bayes formula and how might it apply to the tesla report?'

In [ ]:
output = agent.run(query)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

the forward-looking statements. Factors that might cause such differences include, but are not limited to, those discussed in Part I, Item 1A of this Form 10-K under the heading “Risk Factors,” which are incorporated herein by reference. The following discussion should be read in conjunction with the consolidated financial statements and notes thereto included in Part II, Item 8 of this Form 10-K. All information presented herein is based on the Company’s fiscal calendar. Unless otherwise stated, references to particular years, quarters, months or periods refer to the Company’s fiscal years ended in September and the associated quarters, months and periods of those fiscal years. Each

Llama.generate: 487 prefix-match hit, remaining 27 prompt tokens to eval
llama_perf_context_print:        load time =   46995.00 ms
llama_perf_context_print: prompt eval time =    2334.20 ms /    27 tokens (   86.45 ms per token,    11.57 tokens per second)
llama_perf_context_print:        eval time =   39803.97 ms /   255 runs   (  156.09 ms per token,     6.41 tokens per second)
llama_perf_context_print:       total time =   42479.86 ms /   282 tokens



> Finished chain.

> Finished chain.

> Finished chain.


In [ ]:
output

" Bayes' formula is a mathematical formula for updating the probability of a hypothesis based on new evidence. It states that the posterior probability of a hypothesis (H) is proportional to the product of the likelihood of the evidence given the hypothesis (P(E|H)) and the prior probability of the hypothesis (P(H)): P(H|E) = P(E|H) \\* P(H). In other words, Bayes' formula allows us to update our belief in a hypothesis based on new information or evidence.\nApplying this formula to the Tesla report may involve updating the probability of certain events or scenarios based on new information revealed in the report. For example, if the report reveals that Tesla has entered into a partnership with a major automaker, this could increase the likelihood of the hypothesis that Tesla will be successful in the electric vehicle market. Similarly, if the report reveals that Tesla has made significant advancements in its autonomous driving technology, this could increase the probability of the hypo

In [ ]:
prompt='why would george soros be interested in the tesla report?'

In [ ]:
output = agent.run(query)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

the forward-looking statements. Factors that might cause such differences include, but are not limited to, those discussed in Part I, Item 1A of this Form 10-K under the heading “Risk Factors,” which are incorporated herein by reference. The following discussion should be read in conjunction with the consolidated financial statements and notes thereto included in Part II, Item 8 of this Form 10-K. All information presented herein is based on the Company’s fiscal calendar. Unless otherwise stated, references to particular years, quarters, months or periods refer to the Company’s fiscal years ended in September and the associated quarters, months and periods of those fiscal years. Each

Llama.generate: 513 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =   46995.00 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =   39783.99 ms /   256 runs   (  155.41 ms per token,     6.43 tokens per second)
llama_perf_context_print:       total time =   40118.11 ms /   257 tokens



> Finished chain.

> Finished chain.

> Finished chain.


In [ ]:
output

" Bayes' formula is a mathematical formula for updating the probability of a hypothesis based on new evidence. It states that the posterior probability of a hypothesis (H) given some observed data (D) is equal to the product of the prior probability of the hypothesis (P(H)) and the likelihood of the observed data given the hypothesis (P(D|H)), divided by the evidence (P(D)):\nP(H|D) = P(D|H) \\* P(H) / P(D)\nIn the context of the Tesla report, Bayes' formula could be applied in a few ways:\n1. Update the probability of Tesla's stock price based on new earnings reports or other market data. The prior probability of Tesla's stock price could represent the historical volatility of the stock, while the likelihood of the observed data (e.g. earnings report) given the hypothesis (Tesla's stock price) could be estimated using statistical models such as linear regression or time series analysis.\n2. Evaluate the probability of Tesla's future growth prospects based on new information about the 

In [ ]:
loader = PyPDFLoader('/content/drive/MyDrive/immersion/1987 The Alchemy of Finance  George Soros.pdf')
data3 = loader.load()

In [ ]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

text_chunks = text_splitter.split_documents(data3)

In [ ]:
len(text_chunks)

607

In [ ]:
#get the third chunk
text_chunks[100].page_content

"trinsic growth rates that were rewarded by high multiples. Several \nof the pathbreakers were high-technology companies with a \nstrong defense component whose managements recognized that \ntheir historic growth rate could not be sustained indefinitely: Tex- \ntron, Teledyne, Ling-Temco-Vought (later LTV), to mention a few. \nThey started to acquire more mundane companies, but, as their \nper-share earnings growth accelerated, the multiples expanded \ninstead of contracting. Their success attracted imitators and later \non even the most humdrum companies could attain a high multi- \nple by going on an acquisition spree. For instance, the bulk of \nOgden's earnings was derived from trading scrap metal; neverthe- \nless, the stock sold at more than twenty times earnings at its peak."

In [ ]:
len(text_chunks[2].page_content)

622

In [ ]:
query='why would george soros be interested in the tesla report or why might he think its meaningless?'

In [ ]:
output = agent.run(query)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

included in other income. The combined fair value of these investments was $39 million, $128 million, and $786 million as of the end of fiscal 2002, 2001, and 2000, respectively. The Company believes it is likely there will continue to be significant fluctuations in the fair value of these investments in the future. Further information related to the Company's non-current debt and equity investments may be found in Part II, Item 8 of this Form 10-K at Note 2 of Notes to Consolidated Financial Statements. Factors That May Affect Future Results and Financial Condition Because of the following factors, as well as other factors affecting the Company's operating results and financial cond

Llama.generate: 45 prefix-match hit, remaining 459 prompt tokens to eval
llama_perf_context_print:        load time =   46995.00 ms
llama_perf_context_print: prompt eval time =   46243.37 ms /   459 tokens (  100.75 ms per token,     9.93 tokens per second)
llama_perf_context_print:        eval time =   39629.82 ms /   255 runs   (  155.41 ms per token,     6.43 tokens per second)
llama_perf_context_print:       total time =   86213.57 ms /   714 tokens



> Finished chain.

> Finished chain.

> Finished chain.


In [ ]:
output

" George Soros is a well-known investor and philanthropist who has been involved in various financial markets, including stocks, bonds, currencies, and commodities. However, I don't have any information that suggests he has a specific interest in Tesla or its financial reports. It's possible that he may not be interested in the company or its financials for various reasons, such as:\n1. Diversification of investments: As a successful investor, Soros likely has a diversified portfolio with different asset classes and industries. He might not have a significant interest in Tesla or any other individual stock.\n2. Market volatility: The financial markets can be volatile, and it's possible that Soros may be more focused on broader market trends rather than the performance of a single company like Tesla. He might view the report as unreliable or unnecessary for his investment strategies.\n3. Unfamiliarity with electric vehicle industry: While Tesla is a pioneer in the electric vehicle (EV) 

In [ ]:
loader = PyPDFLoader('/content/drive/MyDrive/immersion/black wednesday.pdf')
data4 = loader.load()

In [ ]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

text_chunks = text_splitter.split_documents(data4)

In [ ]:
len(text_chunks)

4

In [ ]:
#get the third chunk
text_chunks[3].page_content

'certainly not alone in betting against the pound.While he may have borne \na disproportionate amount of the criticism because of his signiﬁcant gains, \nthe government’s own policies are believed by many to have been the root \ncause of the problem, the speculators merely a symptomatic presence. \nThe pound eventually traded as low as 2.16 deutsche  marks in 1995 \nbut then rose as high as 3.44 in 2000 as the Britis h economy recovered \nfrom recession and Germany suffered from the negative effects of euro in- \ntegration. ( See Figure 2.6. ) Some credit today’s strength in the British \neconomy with the interest rate and currency ﬂexibil ity afforded by its po- \nsition outside of the euro system. This is especiall y striking when the \nUnited Kingdom’s economic growth over the last deca de is compared to \nthe growth rates of formerly strong euro area count ries such as Germany \nand France. \nYra Harris amusingly claims in his interview that G reat Britain should \nerect a statue o

In [ ]:
len(text_chunks[2].page_content)

1705

In [ ]:
query='what is black wednesday and how might it apply to the tesla report to george soros?'

In [ ]:
output = agent.run(query)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations The following discussion should be read in conjunction with the consolidated financial statements and accompanying notes included in Part II, Item 8 of this Form 10-K. This section of this Form 10-K generally discusses 2020 and 2019 items and year-to-year comparisons between 2020 and 2019. Discussions of 2018 items and year-to-year comparisons between 2019 and 2018 that are not included in this Form 10-K can be found in “Management’s Discussion and Analysis of Financial Condition and Results of Operations” in Part II, Item 7 of the Company’s Annual Report on Form 10-K for the fiscal year end

Llama.generate: 45 prefix-match hit, remaining 516 prompt tokens to eval
llama_perf_context_print:        load time =   46995.00 ms
llama_perf_context_print: prompt eval time =   53527.04 ms /   516 tokens (  103.73 ms per token,     9.64 tokens per second)
llama_perf_context_print:        eval time =    8967.39 ms /    58 runs   (  154.61 ms per token,     6.47 tokens per second)
llama_perf_context_print:       total time =   62560.57 ms /   574 tokens



> Finished chain.

> Finished chain.

> Finished chain.


In [ ]:
output

' Black Wednesday is not a term used in the Tesla report, nor does it relate to any specific event or date mentioned in the text. Therefore, I cannot provide an answer to this question as there is no information provided in the passage that would support a meaningful response.'

just testing you silly

the idea here is to create a specilaized corpus of knowledge and have the queries return useful information specific to the business proccess.